In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img
from shutil import copy
import os
import cv2 as cv

%matplotlib inline

In [ ]:
# ROOT_DIR = os.curdir

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [ ]:
ROOT_DIR = '/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/'
WORKING_DIR = '/kaggle/working/'

train_dir = os.path.join(WORKING_DIR, 'train')
valid_dir = os.path.join(WORKING_DIR, 'valid')

# for dir_ in [train_dir, valid_dir]:
#     os.makedirs(dir_)
    
train_with_mask = os.path.join(train_dir, 'with_mask')
train_without_mask = os.path.join(train_dir, 'without_mask')
valid_with_mask = os.path.join(valid_dir, 'with_mask')
valid_without_mask = os.path.join(valid_dir, 'without_mask')

for dir_ in [train_with_mask, train_without_mask, valid_with_mask, valid_without_mask]:
    os.makedirs(dir_)

In [ ]:
# copying file from the root directory

for file in os.listdir(os.path.join(ROOT_DIR, 'Train/WithMask')):
    copy(os.path.join(ROOT_DIR, 'Train/WithMask', file), os.path.join(train_with_mask, file))
    
for file in os.listdir(os.path.join(ROOT_DIR, 'Train/WithoutMask')):
    copy(os.path.join(ROOT_DIR, 'Train/WithoutMask', file), os.path.join(train_without_mask, file))

for file in os.listdir(os.path.join(ROOT_DIR, 'Validation/WithMask')):
    copy(os.path.join(ROOT_DIR, 'Validation/WithMask', file), os.path.join(valid_with_mask, file))

for file in os.listdir(os.path.join(ROOT_DIR, 'Validation/WithoutMask')):
    copy(os.path.join(ROOT_DIR, 'Validation/WithoutMask', file), os.path.join(valid_without_mask, file))

In [ ]:
train_mask_dir = os.path.join(WORKING_DIR, 'train')
valid_mask_dir = os.path.join(WORKING_DIR, 'valid')

In [ ]:
len(os.listdir(os.path.join(valid_mask_dir, 'with_mask')))

In [ ]:

def train_val_generator(training_dir, validation_dir):
    """
        Creates the training and validation generators
        
        Args:
        training_dir (string): directory path containing the training images
        validation_dir (string): directory path containing the validation images
        
        returns train_generator, validation_generator
    """
    
    train_datagen = ImageDataGenerator(
                                    rescale=1./255,
                                    rotation_range=60,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest',
                                    featurewise_center=False,
                                    samplewise_center=False,
                                    featurewise_std_normalization=False,
                                    samplewise_std_normalization=False,
                                    zca_whitening=False,
                                    vertical_flip=True
                      )
    
    train_generator = train_datagen.flow_from_directory(training_dir, 
                                                        batch_size=12, 
                                                        class_mode='binary', 
                                                        target_size=(150,150))
    
    valid_datagen = ImageDataGenerator(rescale=1./255,)
    
    validation_generator = valid_datagen.flow_from_directory(validation_dir, 
                                                             batch_size=12, 
                                                             class_mode='binary', 
                                                             target_size=(150, 150))
    
    return train_generator, validation_generator

In [ ]:
train_generator, valid_generator = train_val_generator(train_mask_dir, valid_mask_dir)

In [ ]:
from keras.applications.inception_v3 import InceptionV3

def pretrained_model():
    pretrained_model = InceptionV3(input_shape=(150, 150, 3), weights='imagenet', include_top=False)
    
    for layer in pretrained_model.layers:
        layer.trainable = False
        
    return pretrained_model

In [ ]:
pretrained_model = pretrained_model()

In [ ]:
def output(pretrained_model):
    last_desired_layer = pretrained_model.get_layer('mixed9')
    last_output = last_desired_layer.output
    return last_output

In [ ]:
last_output = output(pretrained_model)

In [ ]:
def create_final_model(pretrained_model, last_output):
    """
        Appends a custom model to a pretrained model
    """
    
    x = layers.Conv2D(64, (3,3), activation='relu', )(last_output)
    x = layers.Flatten()(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=pretrained_model.input, outputs=x)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
model = create_final_model(pretrained_model, last_output)

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

# checkpoint = ModelCheckpoint('InceptionModel', save_best_only=True, verbose = 1)

class Callback(tf.keras.callbacks.Callback):
    
    def on_epoch_end(self, epoch, logs={}):
        if logs['accuracy'] > 0.992 and logs['val_accuracy'] > 0.992:
            self.model.stop_training = True
            
callback = Callback()

In [ ]:
history = model.fit(
    train_generator, 
    validation_data=valid_generator, 
    epochs=40, 
    verbose=1, 
    callbacks=[callback])

In [ ]:
model.save('mask_model')

In [ ]:
!zip -r model.zip /kaggle/working/mask_model

In [ ]:
!ls

In [ ]:
from IPython.display import FileLink
FileLink(r'model.zip')

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(20,8))
ax[0].plot(history.history['accuracy'],)
ax[0].plot(history.history['val_accuracy'])
ax[0].legend(['accuracy', 'val_accuracy'])
ax[1].plot(history.history['loss'], color='g')
ax[1].plot(history.history['val_loss'], color='r')
ax[1].legend(['loss', 'val_loss'])
plt.savefig('plot.png')